### Higher-Order Functions

*Some snippets in this notebook are taken from BLG458E Functional Programming course slides which can be accessed through the following link: https://www.slideshare.net/uyar/tag/blg458e*

**First Class Values** can be assigned, composed with other values, passed to or returned from functions.  

Functions are also first class values! That means they can also be passed as arguments to other functions or returned from functions.

**First-Order Functions** only accepts *data* as parameter and returns data as parameter.  

**Higher-Order Functions** takes other functions as parameter, and returns functions as result

Write a function to sum up the squares in a range

In [1]:
sqr :: Integer -> Integer
sqr a = a*a

-- sum up the squares in a range
sumSqr :: Integer -> Integer -> Integer
sumSqr a b 
  | b < a      = 0
  | otherwise  = sqr a + sumSqr (a+1) b
  
sumSqr 3 5

50

Write a function to sum up the factorials in a range

In [2]:
fac :: Integer -> Integer
fac a 
  | a < 0  = error "invalid arg"
  | a == 0 = 1
  | a > 0  = a * fac (a-1)

-- sum up fractionals in a range
sumFac :: Integer -> Integer -> Integer
sumFac a b 
  | b < a     = 0
  | otherwise = fac a + sumFac (a+1) b
  
sumFac 2 4

32

Have you noticed the pattern? **sumSqr** and **sumFac** are almost identical except the function applied to the list elements.  

Instead of writing almost identical (*therefore redundant*) functions, we can write a generic function and pass the other function, which is applied to each element, as parameter.

In [3]:
-- Function f has the following signature: Integer -> Integer
-- So (Integer -> Integer) part in sumFunc signature comes from function f
-- cuz it's also a parameter to sumFunc

sumFunc :: (Integer -> Integer) -> Integer -> Integer -> Integer
sumFunc f a b
  | b < a     = 0
  | otherwise = f a + sumFunc f (a+1) b

Now we can apply any operation we want! Let's sum up the cubes in a range.

In [4]:
-- first define a function to calculate the cube of a integer
cube :: Integer -> Integer
cube a = a*a*a

-- then pass it to sumFunc. That's it!
sumFunc cube 3 5

216

Remember the function in *IO_operations Notebook* to generate a match?  

In *generateMatch* function, we put a *hardcoded strategy function name*.  
What if we wanted to generate a match using a different strategy?  
In that case, we would need to create a new generateMatch function for every strategy, which causes redundancy!  
Instead, we can pass the strategy function as a parameter to generateMatch function as below.

In [ ]:
generateMatch :: Strategy -> Strategy -> Integer -> Match
generateMatch _ _ 0   = ([], [])
generateMatch sA sB n = step (generateMatch sA sB (n-1))
  where 
    step :: Match -> Match
    step (movesA, movesB) = (sA movesB : movesA, sB movesA : movesB)

### Returning Functions from Other Functions

Now that we have seen how to pass a function to another function as a parameter. Let's now see an example of returning a function from another function. 


**Example**: At Istanbul Technical University, 1st and 2nd year students need to have a minimum GPA of 1.80 and 3rd and 4th year students need to have a minimum GPA of 2.00 in order to avoid academic probation. In the following example, we will declare 3 functions:  
* **probYear1And2**: given gpa, it tells if the student is on probation (for 1st and 2nd year students) 
* **probYear3And4**: given gpa, it tells if the student is on probation (for 3rd and 4th year students)
* **checkProb**: given year, it returns the appropriate probation check function (one of the two given above)

In [6]:
probYear1And2 :: Float -> Bool
probYear1And2 gpa = gpa < 1.80

probYear3And4 :: Float -> Bool
probYear3And4 gpa = gpa < 2.00

checkProb :: Int -> (Float -> Bool)
checkProb year
  | year == 1 || year == 2  = probYear1And2
  | year == 3 || year == 4  = probYear3And4
  | otherwise               = error "Year Error"

Let's check if a 2nd year student with 1.90 gpa is on academic probation or not.

In [7]:
(checkProb 2) 1.90

Line 1: Redundant bracket
Found:
(checkProb 2) 1.9
Why not:
checkProb 2 1.9

False

No! Because the student satisfies the minimum gpa requirement for 2nd year students.  
Let's check if a *4th year* student with 1.90 gpa is on academic probation or not.

In [8]:
(checkProb 4) 1.90

Line 1: Redundant bracket
Found:
(checkProb 4) 1.9
Why not:
checkProb 4 1.9

True

Yes! Because the student does not satisfy the minimum gpa requirement which is at least 2.00.

### Sorting

Let's see an example of insertion sort.

In [9]:
-- function to insert n to a suitable position
ins :: Integer -> [Integer] -> [Integer]
ins n [] = [n]
ins n xs@(x':xs')
  | n <= x'   = n : xs 
  | otherwise = x' : ins n xs'
  
-- main insertion sort function
iSort :: [Integer] -> [Integer]
iSort []     = []
iSort (x:xs) = ins x (iSort xs)

iSort [5,2,8,2,4,6,1,5]

[1,2,2,4,5,5,6,8]

With this implementation, if we want to sort the list in decreasing order, we have to define a new **ins** function.  
Wouldn't it be awesome if we could use ins for both increasing and decreasing order? Indeed it would.  
We will tweak **ins** function so that it will take in an additional function to determine if sorting should be done in inreasing or decreasing order.

In [10]:
-- An additional function argument will determine how to perform
-- the comparison during insertion. It will compare two integers
-- and return a bool, so we add (Integer -> Integer -> Bool) to the signature.

ins2 :: (Integer -> Integer -> Bool)
        -> Integer -> [Integer] -> [Integer]
ins2 p n []   = [n]
ins2 p n xs@(x':xs') 
  | p n x'    = n : xs
  | otherwise = x' : ins2 p n xs'
  
iSort2 :: (Integer -> Integer -> Bool) -> [Integer] -> [Integer]
iSort2 p []     = []
iSort2 p (x:xs) = ins2 p x (iSort2 p xs) 

Now we can use any comparison function we want (<= or >)

In [11]:
iSort2 (<=) [5,2,8,5,3]

[2,3,5,5,8]

In [12]:
iSort2 (>) [5,2,8,5,3]

[8,5,5,3,2]

Let's make another improvement. With the current version of the sorting function, we can only sort a list containing integer values.  
Let's now make the function more generic.

In [13]:
ins3 :: (a -> a -> Bool) -> a -> [a] -> [a]
ins3 p n []   = [n]
ins3 p n xs@(x':xs') 
  | p n x'    = n : xs
  | otherwise = x' : ins3 p n xs'
  
iSort3 :: (a -> a -> Bool) -> [a] -> [a]
iSort3 p []     = []
iSort3 p (x:xs) = ins3 p x (iSort3 p xs) 

In [14]:
iSort3 (<=) ['T', 'I', 'U']

"ITU"

In [15]:
iSort3 (>) ['T', 'I', 'U']

"UTI"

### Lambda Functions
If you are to define a small function which you don't use anywhere else,  
you can define it as an **anonymous function** (aka lambda function)  

Normally we use the following format to call a function:

`f x = e` (f: function name, x: parameter, and e is the expression to be performed)

In case of a lambda function, we use the following format:   

`\param1 param2 .. = expression`

As you see, there's no name for the lambda function, we just pass the parameters.

In [16]:
sumCube :: Integer -> Integer -> Integer
sumCube a b = sumFunc (\x -> x*x*x) a b

sumCube 3 4

91

### Filter
This function is used to select all elements meeting given conditions.

In [17]:
-- Let's find all odd elements in a list
filter odd [1,2,3,4,5,6,7,8,9]

-- this applies odd function to every element in the list

[1,3,5,7,9]

In [18]:
-- let's find even numbers in range [10, 20]
filter even [10 .. 20]

[10,12,14,16,18,20]

In [19]:
-- let's find numbers that are greater than or equal to 5
filter (>= 5) [5,2,7,8,3,12,5,3]

[5,7,8,12,5]

In [20]:
-- find how many elements are greater than a threshold
howManyAbove :: Integer -> [Integer] -> Int
howManyAbove n xs = length (filter (\x -> x >= n) xs)

howManyAbove 6 [4,2,5,7,4,2,13,4,45]

3

**takeWhile**: takes elements from the front of a list while some condition is met.

In [21]:
-- take elements from front while divisible by 3
takeWhile (\x -> x `mod` 3 == 0) [93,30,6,7,8,9, 3]

[93,30,6]

**dropWhile**: drop elements from the front of a list while some condition is met.

In [22]:
-- drop all even elements until you hit an odd element
dropWhile even [4,2,5,4,1]

[5,4,1]

### Map
This function is used to apply a function to all elements in a list

In [23]:
-- floor all elements in a list
map floor [2.3, 4.2, 5.9, 6.6]

[2,4,5,6]

In [24]:
-- convert all elements to string
map show [5, 2, 3, 55]

["5","2","3","55"]

In [25]:
-- convert strings to integers
map (\x -> read x :: Integer) ["1", "4", "50"]

[1,4,50]

In [26]:
-- copy an element n times
copyNtimes :: Int -> a -> [a]
copyNtimes n x = map (\_ -> x) [1 .. n] 

copyNtimes 5 "lol"

["lol","lol","lol","lol","lol"]

**zipWith**: maps two lists over a function  
1st parameter comes from list 1, 2nd parameter comes from list2

In [27]:
zipWith (+) [1,2,3] [6,8,12]

[7,10,15]

In [28]:
zipWith replicate [1,2,3,4,5] "abcde"

["a","bb","ccc","dddd","eeeee"]

### Fold
**foldr1** is used to reduce elements in a list to a single value  

Execution sequence of foldr1 is as follows:  

`foldr1 f [e1, e2, e3, e4]   ->    f e1 (f e2 (f e3 e4)) `

In [29]:
-- sum up all elements in a list
foldr1 (+) [1 .. 10]

Line 1: Use sum
Found:
foldr1 (+)
Why not:
sum

55

*Note: foldr1 doesn't work when the list is empty*  
In that case, use **foldr** and give an initial element in case the list is empty  
It adds this initial element to the end of the list before applying the function.  

Execution sequence of foldr is as follows:  

`foldr f s [e1, e2, e3, e4]   ->    f e1 (f e2 (f e3 (f e4 s))) `

In [30]:
foldr (+) 0 [1,2,3]
foldr (+) 0 []

Line 1: Use sum
Found:
foldr (+) 0
Why not:
sumLine 1: Use sum
Found:
foldr (+) 0
Why not:
sum

6

0

There's also a **foldl** function that starts folding from left.  

`foldl f s [e1, e2, e3, e4]   ->  f (f ( f (f s e1) e2) e3) e4`  

If the function is commutative, foldl and foldr produce the same result, BUT if the function is not commutative, the results are not necessarily the same.

### List Comprehension

* Allows you to create lists quickly.

In [31]:
[2*n | n <- [2,4,7]]  -- equivalent to [2*n for n in [2,4,7]] in Python  

[4,8,14]

In [32]:
[show n | n <- [1 .. 10]]

["1","2","3","4","5","6","7","8","9","10"]

In [33]:
[n | n <- [1 .. 50], even n]  

[2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50]

In [34]:
[n | n <- [1 .. 50], odd n]

[1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49]